In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-28 03:39:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-10-28 03:39:17 (22.0 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open("input.txt",'r',encoding="utf-8") as f:
  text=f.read()

In [ ]:
print("The length of the dataset is:", len(text))

The length of the dataset is: 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars=sorted(list(set(text)))#set(text) creates a set of unique characters
print(set(text))
print(chars)
vocab_size=len(chars)
new_string=''.join(chars)
print(f"The possible characters the model can see: {new_string}")#converts list to string
print(f"The vocab size is: {vocab_size}")

{'M', 'L', 'S', 'O', 'J', 'n', 'a', 'Q', 'p', 'F', 'V', 'd', 'v', 'l', ':', '&', 'f', 'r', 'w', '?', 'N', '!', 'B', 'D', 'Y', '$', 'z', 'x', 't', 'W', 'c', '.', 'E', ',', 'y', 'h', 'X', 'P', '3', 'R', ';', 'K', ' ', 'C', 'i', 'o', '\n', 'G', '-', 'q', "'", 'I', 'H', 'Z', 'j', 'T', 's', 'g', 'b', 'm', 'k', 'A', 'u', 'U', 'e'}
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
The possible characters the model can see: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
The vocab size is: 65


In [ ]:
stoi={ch:i for i,ch in enumerate(chars)}
print(stoi)#string to integer
itos={i:ch for i,ch in enumerate(chars)}
print(itos)#integer to string

def encode(s):
  string_val=[]
  for i in s:
    string_val.append(stoi[i])
  return string_val

def decode(i):
  string_val=""
  for k in i:
    string_val+=(itos[k])
  return string_val


print(encode("hi there"))
print(decode(encode("hi there")))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [ ]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [ ]:
block_size=8

In [ ]:
x=train_data[:block_size]#sliding window
y=train_data[1:block_size+1]
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [ ]:
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f"when input is {context}, output is {target}")

when input is tensor([18]), output is 47
when input is tensor([18, 47]), output is 56
when input is tensor([18, 47, 56]), output is 57
when input is tensor([18, 47, 56, 57]), output is 58
when input is tensor([18, 47, 56, 57, 58]), output is 1
when input is tensor([18, 47, 56, 57, 58,  1]), output is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), output is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), output is 58


In [ ]:
torch.manual_seed(1337)
batch_size=4 #How many independent sequences will we process in parallel?
block_size=8
def get_batch(split):
  #generate a mall batch of data of inputs x and targets y
  data=train_data if split=='train' else val_data
  ix=torch.randint(len(data)-block_size,(batch_size,))#Taking 4 random numbers
  x=torch.stack([data[i:i+block_size] for i in ix]) #  Randomly taking continuous block_size length chunk form the data(batch size(4) of them and stacking)
  y=torch.stack([data[i+1:i+block_size+1] for i in ix] )
  return x,y

In [ ]:
xb,yb=get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("outputs:")
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [ ]:
# BIGRAM EMBEDDING TABLE (vocab_size × vocab_size)

#            Next Token Predictions →
#            a    b    c    d    e   ...
#          ┌─────────────────────────────┐
#       a  │ 0.2  0.5 -0.1  0.3  0.8 ... │ ← Scores for "a→?"
#       b  │ 0.7 -0.3  0.9  0.1  0.4 ... │ ← Scores for "b→?"
# Current c │-0.5  0.6  0.2  0.8 -0.2 ... │ ← Scores for "c→?"
# Token  d │ 0.4  0.1 -0.7  0.5  0.6 ... │ ← Scores for "d→?"
#       e  │ 0.9  0.3  0.4 -0.1  0.2 ... │ ← Scores for "e→?"
#      ... │ ...  ...  ...  ...  ... ... │
#          └─────────────────────────────┘
# ```

# **How it works:**
# ```
# Example: Predict next token after "c"

# 1. Input: token "c" → Look up row 'c'

#    Row c: [-0.5, 0.6, 0.2, 0.8, -0.2, ...]
#            ↑    ↑    ↑    ↑     ↑
#            a    b    c    d     e

# 2. These are logits (raw scores)

# 3. Apply softmax → probabilities:
#    [0.05, 0.15, 0.10, 0.20, 0.06, ...]

# 4. Highest score = 'd' (0.8)
#    → Most likely next token is 'd'
#    → Model predicts: "c" → "d"

# Your data:
# xb = [[18, 47, 56, 57, 58, 1, 15, 47],    # Batch 0: 8 token indices
#       [13, 52, 45, 43, 57, 43, 1, 39],     # Batch 1: 8 token indices
#       [58, 46, 43, 1, 58, 46, 43, 1],      # Batch 2: 8 token indices
#       [39, 52, 41, 43, 1, 58, 46, 43]]     # Batch 3: 8 token indices
# # Shape: (4, 8)

# yb = [[47, 56, 57, 58, 1, 15, 47, 57],    # Targets shifted by 1
#       [52, 45, 43, 57, 43, 1, 39, 58],
#       [46, 43, 1, 58, 46, 43, 1, 39],
#       [52, 41, 43, 1, 58, 46, 43, 1]]
# # Shape: (4, 8)

# # Pass to model:
# out = m(xb, yb)
# ```

# **What happens:**
# ```
# For EACH token index in xb, look up that row in embedding table:

# Batch 0, Position 0: Token 18 → Look up row 18 → Get [65 scores]
# Batch 0, Position 1: Token 47 → Look up row 47 → Get [65 scores]
# Batch 0, Position 2: Token 56 → Look up row 56 → Get [65 scores]
# ...
# Batch 3, Position 7: Token 43 → Look up row 43 → Get [65 scores]
#For each token you get 65 scores

# PAGE 1 (the input xb): 4 rows × 8 columns
# xb = [[18, 47, 56, 57, 58,  1, 15, 47],
#       [13, 52, 45, 43, 57, 43,  1, 39],
#       [58, 46, 43,  1, 58, 46, 43,  1],
#       [39, 52, 41, 43,  1, 58, 46, 43]]
# ```

# **logits = A THICK BOOK with 4 PAGES (one per batch):**

# ---

# **📄 PAGE 1 (Batch 0):** 8 rows × 65 columns
# ```
# Position 0 (token 18): [0.2, 0.5, 0.1, ..., 0.9, ...]  ← 65 scores
# Position 1 (token 47): [0.3, 0.7, 0.2, ..., 0.4, ...]  ← 65 scores
# Position 2 (token 56): [0.1, 0.4, 0.8, ..., 0.6, ...]  ← 65 scores
# Position 3 (token 57): [0.5, 0.2, 0.3, ..., 0.7, ...]  ← 65 scores
# Position 4 (token 58): [0.6, 0.8, 0.1, ..., 0.5, ...]  ← 65 scores
# Position 5 (token  1): [0.4, 0.3, 0.9, ..., 0.2, ...]  ← 65 scores
# Position 6 (token 15): [0.7, 0.1, 0.4, ..., 0.8, ...]  ← 65 scores
# Position 7 (token 47): [0.2, 0.6, 0.5, ..., 0.3, ...]  ← 65 scores
# ```

# ---

# **📄 PAGE 2 (Batch 1):** 8 rows × 65 columns
# ```
# Position 0 (token 13): [0.4, 0.2, 0.7, ..., 0.1, ...]  ← 65 scores
# Position 1 (token 52): [0.8, 0.3, 0.5, ..., 0.6, ...]  ← 65 scores
# Position 2 (token 45): [0.1, 0.9, 0.2, ..., 0.4, ...]  ← 65 scores
# ... (6 more rows)
# ```

# ---

# **📄 PAGE 3 (Batch 2):** 8 rows × 65 columns
# ```
# Position 0 (token 58): [0.6, 0.8, 0.1, ..., 0.5, ...]  ← 65 scores
# Position 1 (token 46): [0.3, 0.5, 0.7, ..., 0.2, ...]  ← 65 scores
# ... (6 more rows)
# ```

# ---

# **📄 PAGE 4 (Batch 3):** 8 rows × 65 columns
# ```
# Position 0 (token 39): [0.5, 0.4, 0.6, ..., 0.9, ...]  ← 65 scores
# Position 1 (token 52): [0.8, 0.3, 0.5, ..., 0.6, ...]  ← 65 scores
# ... (6 more rows)
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):#Lookup table for next token predictor
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits=self.token_embedding_table(idx)
    if targets is None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits,targets)#F.cross entropy needs a 2d logit space and 1d target space

    return logits,loss


#   # Iteration 1:
# idx = [[18, 47, 56]]  # Current: "hel"
# logits = (1, 3, 65)
# logits[:, -1, :] → Use position 2 → Sample token 56 → 'l'
# idx = [[18, 47, 56, 56]]  # Now: "hell"

# # Iteration 2:
# idx = [[18, 47, 56, 56]]  # Current: "hell"
# logits = (1, 4, 65)
# logits[:, -1, :] → Use position 3 (new last) → Sample token 58 → 'o'
# idx = [[18, 47, 56, 56, 58]]  # Now: "hello"

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss=self(idx)
      logits=logits[:,-1,:]
      probs=F.softmax(logits,dim=-1)
      idx_next=torch.multinomial(probs,num_samples=1)#picks max probability value
      idx=torch.cat((idx,idx_next),dim=1)
    return idx

m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits)
print("------------------------------------------------------------------------")
print(loss)
print("------------------------------------------------------------------------")
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)
------------------------------------------------------------------------
tensor(4.8786, grad_fn=<NllLossBackward0>)
------------------------------------------------------------------------

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [ ]:
@torch.no_grad()# We are never gonna call backward on this
def estimate_loss():
    out = {}
    model.eval()

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)

        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()

        out[split] = losses.mean()

    model.train()
    return out

In [ ]:
#creating a Pytorch optimizer
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):
  xb,yb=get_batch('train')
  logits,loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.382369041442871


In [ ]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=300)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


Trick in self attention


In [ ]:
torch.manual_seed(1332)
B,T,C=4,8,2
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1]# all previous elements before the character and including it
    xbow[b,t]=torch.mean(xprev,0)#0th dimension


In [ ]:
x[0]

tensor([[ 0.6828, -0.1091],
        [ 1.6661,  0.0601],
        [-0.3547, -0.3625],
        [-0.4134,  0.5899],
        [ 0.1349,  0.9516],
        [-0.0464, -2.1700],
        [ 0.7447,  1.7339],
        [-0.5819,  0.1811]])

In [ ]:
xbow[0]

tensor([[ 0.6828, -0.1091],
        [ 1.1744, -0.0245],
        [ 0.6647, -0.1372],
        [ 0.3952,  0.0446],
        [ 0.3431,  0.2260],
        [ 0.2782, -0.1733],
        [ 0.3448,  0.0991],
        [ 0.2290,  0.1094]])

In [ ]:
# # Example with T=3
# wei = torch.tril(torch.ones(3, 3))
# # wei = [[1, 0, 0],
# #        [1, 1, 0],
# #        [1, 1, 1]]

# # Sum each row, keepdim=True makes it shape (3, 1)
# row_sums = wei.sum(1, keepdim=True)
# # row_sums = [[1],   <- sum of row 0
# #             [2],   <- sum of row 1
# #             [3]]   <- sum of row 2

# # Division uses broadcasting: each row divided by its own sum
# wei = wei / row_sums
# # Row 0: [1, 0, 0] / 1 = [1.0, 0.0, 0.0]
# # Row 1: [1, 1, 0] / 2 = [0.5, 0.5, 0.0]
# # Row 2: [1, 1, 1] / 3 = [0.33, 0.33, 0.33]wei=torch.tril(torch.ones(T,T))
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x